In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from idtrackerai import ListOfBlobs
from matplotlib.patches import Ellipse
from scipy.interpolate import splev

from fishmidline import get_spline

list_of_blobs = ListOfBlobs.load("100_3.pickle")



Output()

In [18]:
IDENTITY = 100
n_points = 3

In [28]:
midline= np.zeros((19614,100, n_points, 2), dtype=np.float32)
for IDENTITY in range(1,101):
    print(f"Processing IDENTITY={IDENTITY}")
    for frame_number in range(19614):
        blobs_in_frame = list_of_blobs.blobs_in_video[frame_number]
        try:
            blob = [blob for blob in blobs_in_frame if IDENTITY in blob.final_identities][0]
        except IndexError:
            print(f"{IDENTITY=} in {frame_number=} not found!")
            continue

        spline_params = get_spline(blob)

        spline_samples = np.linspace(0, 1, n_points)
        midline_x, midline_y = splev(spline_samples, spline_params)
        head = splev(0, spline_params)
        derivative_x, derivative_y = splev(spline_samples, spline_params, der=1)
        midline_angles = np.arctan2(derivative_x, derivative_y)

        #center, (width, height), angle = cv2.fitEllipse(blob.contour)
        #center = np.asarray(center) - blob.bbox_in_frame_coordinates[0] + 1
        midline[frame_number,IDENTITY-1, :, 0] = midline_x+blob.bbox_in_frame_coordinates[0][0]
        midline[frame_number,IDENTITY-1, :, 1] = midline_y+blob.bbox_in_frame_coordinates[0][1]

    """
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 3, 1)
    ax2 = fig.add_subplot(1, 3, 2)
    ax3 = fig.add_subplot(1, 3, 3)

    binary_image = blob.get_bbox_mask()
    for ax in (ax1, ax2, ax3):
        ax.set(aspect=1, xticks=(), yticks=())
        ax.imshow(binary_image, cmap="gray")

    ax1.add_artist(
        Ellipse(
            xy=tuple(center),
            width=width,
            height=height,
            angle=angle,
            facecolor="none",
            edgecolor="red",
        )
    )
    ax1.plot(*(np.asarray(blob.centroid) - blob.bbox_in_frame_coordinates[0] + 1), "g.")
    ax1.plot(*(np.asarray(center)), "r.")
    ax1.set(title="Fitted ellipse\nand centroid")

    ax2.plot(midline_x, midline_y, ".-")
    ax2.plot(*splev(0, spline_params), "r.")
    ax2.set(title="Midline and\nhead position")

    ax3.plot(midline_x, midline_y, ".")
    ax3.plot(
        [
            midline_x - 2 * np.sin(midline_angles),
            midline_x + 2 * np.sin(midline_angles),
        ],
        [
            midline_y - 2 * np.cos(midline_angles),
            midline_y + 2 * np.cos(midline_angles),
        ],
        "r-",
        lw=1,
    )
    ax3.set(title="Midline angles")
    fig.tight_layout()
    fig.savefig(f"frame_{frame_number}.png")
    plt.close()
    """
np.save(f"data_midline/100_2", midline)

Processing IDENTITY=1


/tmp/ipykernel_143197/758597494.py:22: DeprecationWarning: Call to deprecated function (or staticmethod) bbox_in_frame_coordinates. (Use `bbox_corners` instead) -- Deprecated since version 6.0.0.
  midline[frame_number,IDENTITY-1, :, 0] = midline_x+blob.bbox_in_frame_coordinates[0][0]
/tmp/ipykernel_143197/758597494.py:23: DeprecationWarning: Call to deprecated function (or staticmethod) bbox_in_frame_coordinates. (Use `bbox_corners` instead) -- Deprecated since version 6.0.0.
  midline[frame_number,IDENTITY-1, :, 1] = midline_y+blob.bbox_in_frame_coordinates[0][1]


IndexError: list index out of range